In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch

# import
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings

from langchain import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from transformers import AutoTokenizer, AutoModel, AutoConfig
from typing import Any, Dict, List, Mapping, Optional, Tuple, Union
from torch.mps import empty_cache
import torch
from langchain.chains import RetrievalQA

torch.manual_seed(1234)


In [2]:

class Qwen:
    def __init__(self, model_path: str, device: str = "cuda:1"):
        self.model_path = model_path
        self.device = device
        self.tokenizer = None
        self.model = None

    def load_model(self):
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(self.model_path, trust_remote_code=True)
        self.model.to(self.device)
        self.model.eval()
        self.model.generation_config = GenerationConfig.from_pretrained(self.model_path, trust_remote_code=True)

    def generate_response(self, image_folder: str, image_index: int, prompt: str, history: list = None):
        query = self.tokenizer.from_list_format([
            {'image': f'{image_folder}/{image_index}.png'}, 
            {'text': prompt}
        ])

        # Ensure tensors are on the correct device if necessary
        if isinstance(query, torch.Tensor):
            query = query.to(self.device)

        response, history = self.model.chat(self.tokenizer, query=query, history=history)
        return response, history

# Path to the model directory
model_path = "/data1/dxw_data/llm/Qwen-VL-Chat"
# Specify the device (e.g., 'cuda:1', 'cuda:1')
device = 'cuda:1'

# Instantiate and load the model
qwen_model = Qwen(model_path, device)
qwen_model.load_model()

2024-06-15 17:25:06.258986: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-15 17:25:06.381415: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-15 17:25:06.948468: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2024-06-15 17:25:06.948540: W tensorflow/compiler/xla/stream_exec

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [3]:

# Example usage
image_folder = "/data1/dxw_data/llm/tiktok/data/output_comments"
image_index = 0  # Change this to the actual image index
# likes:
# prompt = "Recognize and output all numbers in the image, possibly with the Chinese character '万', if possible."
# comments:
prompt = "描述这个图片"
history = []


response, history = qwen_model.generate_response(image_folder, image_index, prompt, history)
print(response)
print(history)

图中是一个身穿蓝色裙子的女性站在一个类似桥的地方。
[('Picture 1:<img>/data1/dxw_data/llm/tiktok/data/output_comments/0.png</img>\n描述这个图片', '图中是一个身穿蓝色裙子的女性站在一个类似桥的地方。')]


In [ ]:
# ------------------------------------ 更多测试

In [4]:
import os
import json

def generate_responses(image_folder, start_index, end_index, prompt, output_json):
    responses = {}
    history = []  # Initialize history

    for image_index in range(start_index, end_index + 1):
        # Generate response with history
        response, updated_history = qwen_model.generate_response(image_folder, image_index, prompt, history)
        responses[image_index] = response
        print(f"Image index {image_index} response: {response}")

        # Update the history
        history.append((f"Picture {image_index}:<img>{image_folder}/{image_index}.png</img>\n{prompt}", response))

        # Keep only the last 2 entries in the history
        if len(history) > 1:
            history = history[-1:]

    # Save responses to JSON file
    with open(output_json, 'w', encoding='utf-8') as json_file:
        json.dump(responses, json_file, ensure_ascii=False, indent=4)



In [5]:
# 使用示例
image_folder = "/data1/dxw_data/llm/tiktok/data/output_likes"
start_index = 0
end_index = 55
prompt = "识别图中的5个数字。注意如果没有明显数字，就输出[0,0,0,0,0]"
output_json = "/data1/dxw_data/llm/tiktok/data/output_txt/responses_likes.json"

generate_responses(image_folder, start_index, end_index, prompt, output_json)

Image index 0 response: 图中可能隐藏的数字有：2787，226，387，220，1771。
Image index 1 response: 图中可能隐藏的数字有：2787，226，387，220，1771。
Image index 2 response: 图中可能隐藏的数字有：2787，226，387，220，1771。
Image index 3 response: 图中可能隐藏的数字有：2787，226，387，220，1771。
Image index 4 response: 图中可能隐藏的数字有：45，22，71，20，86。
Image index 5 response: 图中可能隐藏的数字有：45，22，71，20，86。
Image index 6 response: 图中可能隐藏的数字有：45，22，71，20，86。
Image index 7 response: 图中可能隐藏的数字有：46，71，20，86，1657。
Image index 8 response: 图中可能隐藏的数字有：46，71，20，86，1657。
Image index 9 response: 图中可能隐藏的数字有：46，71，20，86，1657。
Image index 10 response: 图中可能隐藏的数字有：46，71，20，86，1657。
Image index 11 response: 图中可能隐藏的数字有：10，100，1000，10000，100000。
Image index 12 response: 图中可能隐藏的数字有：10，100，1000，10000，100000。
Image index 13 response: 图中可能隐藏的数字有：10，100，1000，10000，100000。
Image index 14 response: 图中可能隐藏的数字有：10，100，1000，10000，100000。
Image index 15 response: 图中可能隐藏的数字有：158，159，160，167，177。
Image index 16 response: 图中可能隐藏的数字有：1375，1875，2675，1875，2675。
Image index 17 response: 图中可能隐藏的数字有

In [7]:
# 使用示例
image_folder = "/data1/dxw_data/llm/tiktok/data/output_comments"
start_index = 0
end_index = 55
prompt = "识别图片中的所有文本"
output_json = "/data1/dxw_data/llm/tiktok/data/output_txt/responses_comments.json"

generate_responses(image_folder, start_index, end_index, prompt, output_json)

Image index 0 response: 图中包含多个文本，包括：

- 香港迪士尼乐园
- 艾莎女王：你不要给我哇乱叫！
- #香港迪士尼魔雪奇缘世界
- #展开
- #香港迪士尼冰雪奇缘园区
- 相关爆梗：烟花薯片
Image index 1 response: 图中包含多个文本，包括：

- 香港迪士尼乐园
- 艾莎女王：你不要给我哇乱叫！
- #香港迪士尼魔雪奇缘世界
- #展开
- #香港迪士尼冰雪奇缘园区
- 相关爆梗：烟花薯片
Image index 2 response: 图中包含多个文本，包括：

- 香港迪士尼乐园
- 艾莎女王：你不要给我哇乱叫！
- #香港迪士尼魔雪奇缘世界
- #展开
- #香港迪士尼冰雪奇缘园区
- 相关爆梗：烟花薯片
Image index 3 response: 图中包含多个文本，包括：

- 天猫-徐芬
- 体芬科技电动牙刷，三维十级调刷，创造健康清洁口腔，智能
- 就“刷”难求
- 科技，创造健康清洁口腔，智能
Image index 4 response: 图中包含多个文本，包括：

- @奶芙芙
- 大清早就在旁边喝喝喝的要饭了
- #心态全写在脸上的猫猫#小奶猫
- 热搜：猫猫有多迷人 性感37993万人在看
Image index 5 response: 图中包含多个文本，包括：

- @奶芙芙
- 大清早就在旁边喝喝喝的要饭了
- #心态全写在脸上的猫猫#小奶猫
- 热搜：猫猫有多迷人 性感37993万人在看
Image index 6 response: 图中包含多个文本，包括：

- @奶芙芙
- 大清早就在旁边喝喝喝的要饭了
- #心态全写在脸上的猫猫#小奶猫
- 热搜：猫猫有多迷人 性感37993万人在看
Image index 7 response: 图中包含多个文本，包括：

- 吹你手还挺小的
- @肉脸静怡酱 《班级里的某些女43》#剧情#校园
- 导
- 4.6万
- 1.3万
- 肉脸静怡酱
- 班级里的某些女43
- #剧情#校园
Image index 8 response: 图中包含多个文本，包括：

- 好像还沒我没手大
- @肉脸静怡酱 《班级里的某些女43》#剧情#校园
- 导
- 4.6万
- 1.3万
- 肉脸静怡酱
- 班级里的某些女43
- #剧情#校园
Im

In [8]:
# 使用示例
image_folder = "/data1/dxw_data/llm/tiktok/data/output_caption"
start_index = 0
end_index = 55
prompt = "描述这个图片"
output_json = "/data1/dxw_data/llm/tiktok/data/output_txt/responses_caption.json"

generate_responses(image_folder, start_index, end_index, prompt, output_json)

Image index 0 response: 图中是一个类似于冰雪奇缘中的场景，一个蓝眼睛的女性站在一个魔法桥上，桥的背景是黑色的，女性穿着蓝色的裙子，场景非常梦幻。
Image index 1 response: 图中是一个类似于冰雪奇缘中的场景，一个蓝眼睛的女性站在一个魔法桥上，桥的背景是黑色的，女性穿着蓝色的裙子，场景非常梦幻。
Image index 2 response: 图中是一个类似于冰雪奇缘中的场景，一个蓝眼睛的女性站在一个魔法桥上，桥的背景是黑色的，女性穿着蓝色的裙子，场景非常梦幻。
Image index 3 response: 图中是一个抖音短视频的截图，视频中是一个电动牙刷的广告宣传页面，下面有一个红色的评论区，其中有一条评论是“就一‘刷’难求”
Image index 4 response: 图是一张猫咪的短视频截图，背景是白色的，猫咪正在床上睡觉，床单是白色的。视频的评论区有留言，有点赞，播放量已经达到了一百万。
Image index 5 response: 图是一张猫咪的短视频截图，背景是白色的，猫咪正在床上睡觉，床单是白色的。视频的评论区有留言，有点赞，播放量已经达到了一百万。
Image index 6 response: 图是一张猫咪的短视频截图，背景是白色的，猫咪正在床上睡觉，床单是白色的。视频的评论区有留言，有点赞，播放量已经达到了一百万。
Image index 7 response: 图中是一个身穿学生装的女生在进行直播，背景是白色的，她手上还戴着一个蓝色的手表。
Image index 8 response: 图中是一个身穿学生装的女生在进行直播，背景是白色的，她手上还戴着一个蓝色的手表。
Image index 9 response: 图中是一个身穿学生装的女生在进行直播，背景是白色的，她手上还戴着一个蓝色的手表。
Image index 10 response: 图中是一个身穿学生装的女生在进行直播，背景是白色的，她手上还戴着一个蓝色的手表。
Image index 11 response: 图中是一个手机截图，截图中有一个抖音视频的点赞评论等信息，视频标题是“《班级里的某些女生43》#剧情#校园”，视频的点赞数量是16.5万，评论数量是1.3万，其中用户名字有“肉脸静静酱”、“梦里很丧”等。
Image ind

In [9]:
# 使用示例
image_folder = "/data1/dxw_data/llm/tiktok/data/output_title"
start_index = 0
end_index = 55
prompt = "识别图片中的所有文本"
output_json = "/data1/dxw_data/llm/tiktok/data/output_txt/responses_title.json"

generate_responses(image_folder, start_index, end_index, prompt, output_json)

Image index 0 response: 图中包含多个文本，包括：

- 香港迪士尼乐园
- 艾莎女王：你不要给我哇哇乱叫！
- #香港迪士尼 魔雪奇缘世界...
- 展开
- #香港迪士尼 魔雪奇缘世界...
- 展开
- 香港迪士尼乐园酒店 限时秒杀
Image index 1 response: 图中包含多个文本，包括：

- 香港迪士尼乐园
- 艾莎女王：你不要给我哇哇乱叫！
- #香港迪士尼 魔雪奇缘世界...
- 展开
- #香港迪士尼 魔雪奇缘世界...
- 展开
- 香港迪士尼乐园酒店 限时秒杀
Image index 2 response: 图中包含多个文本，包括：

- 香港迪士尼乐园
- 艾莎女王：你不要给我哇哇乱叫！
- #香港迪士尼 魔雪奇缘世界...
- 展开
- #香港迪士尼 魔雪奇缘世界...
- 展开
- 香港迪士尼乐园酒店 限时秒杀
Image index 3 response: 图中包含多个文本，包括：

- 天猫-徐芬
- 徐芬科技电动牙刷，
- 三维十调控，
- 智能科技，
- 创建健康清洁口腔
- 广告
Image index 4 response: 图中包含多个文本，包括：

- 奶芙芙
- 大清早就守在旁邊哼唱哪個要吃饭
- #心態全寫在臉上的貓貓 #奶奶
- 热搜：猫猫有多迷人性 13799.3萬人在看
Image index 5 response: 图中包含多个文本，包括：

- 奶芙芙
- 大清早就守在旁邊哼唱哪個要吃饭
- #心態全寫在臉上的貓貓 #奶奶
- 热搜：猫猫有多迷人性 13799.3萬人在看
Image index 6 response: 图中包含多个文本，包括：

- 奶芙芙
- 大清早就守在旁邊哼唱哪個要吃饭
- #心態全寫在臉上的貓貓 #奶奶
- 热搜：猫猫有多迷人性 13799.3萬人在看
Image index 7 response: 图中包含多个文本，包括：

- 收你一逃挺小的
- 肉脸静怡著《班级里的某些女生43》#剧情感#校园
- 1.3万
- 16.5万
Image index 8 response: 图中包含多个文本，包括：

- 好惨还没我惨大
- 肉脸静怡著《班级里的某些女生43》#剧情感#校园
- 1.3万
- 16.5万
Image index 9 respons